In [2]:
from IPython import get_ipython
from IPython.display import display
from keras import __version__ # Import keras version
tf.keras.__version__ = __version__

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Embedding, Flatten, Concatenate, Dropout
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:
df = pd.read_csv("my_dataset.csv")
print(df["Soil"].unique())
print(df["Crop"].unique())

['Loamy' 'Clay' 'Sandy' 'Black']
['cotton' 'orange' 'wheat' 'maize' 'rice' 'potato' 'tomato' 'carrot'
 'cabbage' 'banana' 'onion' 'pepper' 'lettuce' 'sunflower' 'soybean'
 'tobacco' 'sugarcane' 'peanut' 'coffee' 'tea']


In [25]:
print(df['Fertilizer'].value_counts())


Fertilizer
NPK 14-14-14    13191
DAP              8148
Urea             4882
MOP              3779
Name: count, dtype: int64


In [5]:
soil_encoder = LabelEncoder()
crop_encoder = LabelEncoder()
df["Soil"] = soil_encoder.fit_transform(df["Soil"])
df["Crop"] = crop_encoder.fit_transform(df["Crop"])

soil_mapping = dict(zip(soil_encoder.classes_, range(len(soil_encoder.classes_))))
print(soil_mapping)

{'Black': 0, 'Clay': 1, 'Loamy': 2, 'Sandy': 3}


In [6]:
scaler = StandardScaler()
df[["Temperature", "Nitrogen", "Phosphorus", "Potassium"]] = scaler.fit_transform(df[["Temperature", "Nitrogen", "Phosphorus", "Potassium"]])

fertilizer_encoder = LabelEncoder()
df["Fertilizer"] = fertilizer_encoder.fit_transform(df["Fertilizer"])

In [8]:
X = df[["Temperature", "Soil", "Nitrogen", "Phosphorus", "Potassium", "Crop"]].values
y = df["Fertilizer"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [9]:
fert_class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train
)
fert_class_weights = {i: w * 3 for i, w in enumerate(fert_class_weights)}

In [10]:
input_layer = Input(shape=(6,))
x = Dense(256, activation="relu")(input_layer)  # More units
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)

# Fertilizer prediction branch
fert_branch = Dense(64, activation='relu')(x)
fert_branch = Dropout(0.2)(fert_branch)
fertilizer_output = Dense(
    len(fertilizer_encoder.classes_),
    activation="softmax",
    name="fertilizer_output"
)(fert_branch)

model = Model(inputs=input_layer, outputs=fertilizer_output)

In [11]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
unique_classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=unique_classes, y=y_train)
fert_class_weights = dict(zip(unique_classes, class_weights))

In [12]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# ======= MODEL TRAINING =======
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    class_weight=fert_class_weights
)

# ======= EVALUATION =======
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nFinal Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6508 - loss: 0.7512 - val_accuracy: 0.8163 - val_loss: 0.4423
Epoch 2/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7952 - loss: 0.4392 - val_accuracy: 0.8203 - val_loss: 0.4217
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8111 - loss: 0.4012 - val_accuracy: 0.8282 - val_loss: 0.4047
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8237 - loss: 0.3740 - val_accuracy: 0.8167 - val_loss: 0.4221
Epoch 5/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8218 - loss: 0.3619 - val_accuracy: 0.8297 - val_loss: 0.3803
Epoch 6/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8265 - loss: 0.3630 - val_accuracy: 0.8300 - val_loss: 0.3871
Epoch 7/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8284 - loss: 0.3561 - val_accuracy: 0.8382 - val_loss: 0.3576
Epoch 8/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8356 - loss: 0.3379 - val_accuracy: 0.

In [13]:
def predict_fertilizer(raw_input):
    """Simplified prediction function with proper feature handling"""
    # Extract features in order: [Temp, Soil, N, P, K, Crop]
    temp = raw_input[0]
    soil = raw_input[1]
    n = raw_input[2]
    p = raw_input[3]
    k = raw_input[4]
    crop = raw_input[5].lower()

    # Process numerical features (Temp, N, P, K)
    num_features = np.array([[temp, n, p, k]])
    scaled_num = scaler.transform(num_features)

    # Process categorical features (Soil, Crop)
    encoded_soil = soil_encoder.transform([soil])[0]
    encoded_crop = crop_encoder.transform([crop])[0]

    # Create final input array in correct order
    model_input = np.array([[
        scaled_num[0][0],  # Scaled Temperature
        encoded_soil,      # Encoded Soil
        scaled_num[0][1],  # Scaled Nitrogen
        scaled_num[0][2],  # Scaled Phosphorus
        scaled_num[0][3],  # Scaled Potassium
        encoded_crop       # Encoded Crop
    ]])

    # Make prediction
    pred = model.predict(model_input)
    return fertilizer_encoder.inverse_transform([np.argmax(pred)])[0]

In [14]:
model.save('my_model.keras')

In [15]:
import joblib
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(soil_encoder, 'soil_encoder.pkl')
joblib.dump(crop_encoder, 'crop_encoder.pkl')
joblib.dump(fertilizer_encoder, 'fertilizer_encoder.pkl')

['fertilizer_encoder.pkl']

In [21]:
sample = [28.5, "Black", 10, 70, 90, "Wheat"]
print(predict_fertilizer(sample))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Urea


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
